In [ ]:
!nvidia-smi

Mon Apr 18 07:31:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pwd

/content


In [ ]:
!ls

sample_data  test_shrutime.csv	train_shrutime.csv


In [ ]:
! pip install pytorch-tabnet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(0)


import os
from pathlib import Path
import shutil
import gzip

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('train_shrutime.csv')
n_total = len(df)

# Train, val and test split follows
# Rory Mitchell, Andrey Adinets, Thejaswi Rao, and Eibe Frank.
# Xgboost: Scalable GPU accelerated learning. arXiv:1806.11248, 2018.

# train_val_indices, test_indices = train_test_split(
#     range(n_total), test_size=0.2, random_state=0)
# train_indices, val_indices = train_test_split(
#     train_val_indices, test_size=0.2 / 0.6, random_state=0)

# traindf = df.iloc[train_indices]
# valdf = df.iloc[val_indices]
# testdf = df.iloc[test_indices]
# traindf = traindf.sample(frac=1)

# traindf.to_csv('train_shrutime.csv', index=False, header=False)
# valdf.to_csv('val_shrutime.csv', index=False, header=False)
# testdf.to_csv('test_shrutime.csv', index=False, header=False)

In [ ]:
test_df=pd.read_csv('test_shrutime.csv')

In [ ]:
def convert_to_one_hot(y):
    seen = False
    for i in range(len(y)):
        l = [0 for i in range(7)]
        l[y[i] - 1] = 1
        if not seen:
            y_o = np.array(l.copy())
            seen = True
        else:
            y_o = np.vstack([y_o, l.copy()])
    return y_o
# data = pd.read_csv(input_dir + 'Churn_Modelling.csv')
# data = data.drop(["RowNumber", "CustomerId", "Surname"], axis = 1)


def convert_country(y):
    seen = False
    for i in range(len(y)):
        if not seen:
            y_o = np.array([m1[y[i]]])
            seen = True
        else:
            y_o = np.vstack([y_o, [m1[y[i]]]])
    return y_o

def convert_gender(y):
    seen = False
    for i in range(len(y)):
        if not seen:
            y_o = np.array([m2[y[i]]])
            seen = True
        else:
            y_o = np.vstack([y_o, [m2[y[i]]]])
    return y_o
    


In [ ]:
!pip install fast_ml

     |████████████████████████████████| 42 kB 833 kB/s 


In [ ]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography,Gender
0,708,67,1,0.00,2,0,1,3837.08,0,0,1
1,603,35,1,123407.69,1,1,0,152541.89,1,2,0
2,618,33,4,0.00,2,1,1,77550.18,0,1,0
3,592,37,9,0.00,3,1,1,10656.89,0,1,0
4,517,59,8,154110.99,2,1,0,101240.08,1,2,1


In [ ]:
m1 = {"France" : 0, "Spain" : 1, "Germany" : 2}
m2 = {"Male" : 0, "Female" : 1}

# y = df["Geography"]
# y = y.to_numpy()
# y = convert_country(y)
# df = df.drop("Geography", axis = 1)
# df["Geography"] = y

# y = df["Gender"]
# y = y.to_numpy()
# y = convert_gender(y)
# df = df.drop("Gender", axis = 1)
y=df["Exited"]
y_test=test_df["Exited"]
df = df.drop("Exited", axis = 1)
test_df = test_df.drop("Exited", axis = 1)
# y_test = test_df["Geography"]
# y_test = y_test.to_numpy()
# y_test = convert_country(y_test)
# test_df = test_df.drop("Geography", axis = 1)
# test_df["Geography"] = y_test

# y_test = test_df["Gender"]
# y_test = y_test.to_numpy()
# y_test = convert_gender(y_test)
# test_df = test_df.drop("Gender", axis = 1)
# test_df["Gender"] = y_test
# print(test_df.shape)

# x_train = train.drop("Cover_Type", axis = 1)
# y_train = train[train.columns[-1]]
from sklearn.model_selection import train_test_split

x_train,x_valid,y_train,y_valid=train_test_split(df,y,train_size=0.9)

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
#y_train = np.expand_dims(y_train, axis=1)
#y_train = convert_to_one_hot(y_train)


x_valid = x_valid.to_numpy()
y_valid = y_valid.to_numpy()
#y_valid = np.expand_dims(y_valid, axis=1)
#y_valid = convert_to_one_hot(y_valid)

x_test = test_df.to_numpy()
y_test = y_test.to_numpy()
#y_test = np.expand_dims(y_test, axis=1)
#y_test = convert_to_one_hot(y_test)

print(x_train.shape)
print(y_train.shape)

print(x_valid.shape)
print(y_valid.shape)

print(x_test.shape)
print(y_test.shape)


print(df.shape)
y = df[df.columns[-1]]
y = y.to_numpy()
target_cols = ["Exited"]

(7200, 10)
(7200,)
(800, 10)
(800,)
(2000, 10)
(2000,)
(8000, 10)


In [ ]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography,Gender
0,708,67,1,0.00,2,0,1,3837.08,0,1
1,603,35,1,123407.69,1,1,0,152541.89,2,0
2,618,33,4,0.00,2,1,1,77550.18,1,0
3,592,37,9,0.00,3,1,1,10656.89,1,0
4,517,59,8,154110.99,2,1,0,101240.08,2,1


In [ ]:
x_train

array([[6.2600000e+02, 3.9000000e+01, 1.0000000e+01, ..., 5.1467920e+04,
        2.0000000e+00, 0.0000000e+00],
       [7.4800000e+02, 4.0000000e+01, 0.0000000e+00, ..., 6.0416760e+04,
        0.0000000e+00, 0.0000000e+00],
       [7.8600000e+02, 3.4000000e+01, 3.0000000e+00, ..., 1.8368209e+05,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [5.8100000e+02, 5.0000000e+01, 4.0000000e+00, ..., 8.0701720e+04,
        1.0000000e+00, 1.0000000e+00],
       [6.4500000e+02, 4.5000000e+01, 8.0000000e+00, ..., 2.2558740e+04,
        0.0000000e+00, 1.0000000e+00],
       [5.7300000e+02, 3.7000000e+01, 6.0000000e+00, ..., 1.9399537e+05,
        0.0000000e+00, 1.0000000e+00]])

In [ ]:
y_train[0]

1

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

In [ ]:
from hyperopt import hp

In [ ]:
import hyperopt

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer

from hyperopt import fmin, tpe, Trials, STATUS_OK, STATUS_FAIL
from hyperopt.pyll.base import scope
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
import torch.nn as nn

In [ ]:
def tb_ll_cv(space):
  #print("hello")
  model=TabNetClassifier(n_a=int(space['n_a']), n_d=int(space['n_a']), n_steps=int(space['n_steps']), momentum=space['momentum'], gamma=space['gamma'], optimizer_params=dict(lr=space['lr']), device_name='cuda')
  model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], batch_size=int(space['batch_size']),max_epochs=100,eval_metric=['logloss'])
  #print(model.history["val_0_logloss"])
  return min(model.history["val_0_logloss"])


In [ ]:
reg_params = {'n_a': hp.quniform('feature_dim', 20, 60, 1),'n_steps': hp.quniform('n_steps', 1, 8, 1),'momentum': hp.uniform('momentum', np.exp(-5), np.exp(-1)),'gamma': hp.uniform('relaxation_factor', 0.3, 2),
    'batch_size': hp.choice('batch_size',[512, 1024, 2048, 4096, 8192]), 'lr':hp.loguniform('learning_rate', -5, 0)
}

In [ ]:
trials=Trials()
best=fmin(fn=tb_ll_cv, space=reg_params, algo=tpe.suggest, max_evals=15, trials=trials)

Device used : cuda
epoch 0  | loss: 3.2162  | val_0_logloss: 3.99704 |  0:00:00s
epoch 1  | loss: 0.60427 | val_0_logloss: 1.85831 |  0:00:01s
epoch 2  | loss: 0.54808 | val_0_logloss: 0.70269 |  0:00:01s
epoch 3  | loss: 0.52125 | val_0_logloss: 0.88212 |  0:00:02s
epoch 4  | loss: 0.47213 | val_0_logloss: 0.81193 |  0:00:03s
epoch 5  | loss: 0.45997 | val_0_logloss: 0.9907  |  0:00:03s
epoch 6  | loss: 0.43664 | val_0_logloss: 0.86119 |  0:00:04s
epoch 7  | loss: 0.44199 | val_0_logloss: 1.63431 |  0:00:04s
epoch 8  | loss: 0.45031 | val_0_logloss: 1.52461 |  0:00:05s
epoch 9  | loss: 0.42584 | val_0_logloss: 1.6097  |  0:00:05s
epoch 10 | loss: 0.41859 | val_0_logloss: 0.59092 |  0:00:06s
epoch 11 | loss: 0.41197 | val_0_logloss: 0.66818 |  0:00:07s
epoch 12 | loss: 0.411   | val_0_logloss: 0.86109 |  0:00:07s
epoch 13 | loss: 0.4114  | val_0_logloss: 0.51552 |  0:00:08s
epoch 14 | loss: 0.41349 | val_0_logloss: 0.63898 |  0:00:08s
epoch 15 | loss: 0.41389 | val_0_logloss: 0.59316 |

In [ ]:
a1=hyperopt.pyll.stochastic.sample(reg_params)
print(a1)
# tb_ll_cv(a1)
# #torch.from_numpy(a1['gamma']).dtype

{'batch_size': 512, 'gamma': 1.6867551228669113, 'lr': 0.952292858193817, 'momentum': 0.03044791321571015, 'n_a': 21.0, 'n_steps': 7.0}


In [ ]:
best

{'batch_size': 0,
 'feature_dim': 27.0,
 'learning_rate': 0.04732037402888312,
 'momentum': 0.28382137773095045,
 'n_steps': 6.0,
 'relaxation_factor': 0.9003863490475343}

In [ ]:
batch_size_list=[512, 1024, 2048, 4096, 8192]

In [ ]:
model=TabNetClassifier(n_a=int(best['feature_dim']), n_d=int(best['feature_dim']), n_steps=int(best['n_steps']), momentum=best['momentum'], gamma=best['relaxation_factor'], optimizer_params=dict(lr=best['learning_rate']), device_name='cuda')
model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], batch_size=batch_size_list[best['batch_size']],max_epochs=100,eval_metric=['logloss'])

Device used : cuda
epoch 0  | loss: 0.70165 | val_0_logloss: 2.469   |  0:00:01s
epoch 1  | loss: 0.4171  | val_0_logloss: 3.01514 |  0:00:02s
epoch 2  | loss: 0.40111 | val_0_logloss: 0.88182 |  0:00:03s
epoch 3  | loss: 0.38686 | val_0_logloss: 1.0809  |  0:00:04s
epoch 4  | loss: 0.36489 | val_0_logloss: 0.70463 |  0:00:05s
epoch 5  | loss: 0.35623 | val_0_logloss: 0.68079 |  0:00:06s
epoch 6  | loss: 0.35406 | val_0_logloss: 0.62304 |  0:00:07s
epoch 7  | loss: 0.34953 | val_0_logloss: 0.56472 |  0:00:08s
epoch 8  | loss: 0.35473 | val_0_logloss: 0.49148 |  0:00:10s
epoch 9  | loss: 0.34656 | val_0_logloss: 0.46911 |  0:00:11s
epoch 10 | loss: 0.3412  | val_0_logloss: 0.45142 |  0:00:12s
epoch 11 | loss: 0.33956 | val_0_logloss: 0.35024 |  0:00:13s
epoch 12 | loss: 0.34089 | val_0_logloss: 0.40366 |  0:00:14s
epoch 13 | loss: 0.3483  | val_0_logloss: 0.36438 |  0:00:15s
epoch 14 | loss: 0.34671 | val_0_logloss: 0.34804 |  0:00:16s
epoch 15 | loss: 0.33942 | val_0_logloss: 0.32916 |

In [ ]:
criterion = nn.CrossEntropyLoss()
loss1 = criterion(torch.from_numpy(model.predict_proba(x_test)),torch.from_numpy(y_test))
print(loss1)

tensor(0.4686)


In [ ]:
l=[]
for i in torch.from_numpy(model.predict_proba(x_test)):
  l.append(i[0])
b=torch.FloatTensor(l)

In [ ]:
criterion = nn.BCELoss()
loss1 = criterion(1-b,torch.from_numpy(y_test).float())
print(loss1)

tensor(0.3552)


In [ ]:
a=model.predict_proba(x_test)

In [ ]:
pd.DataFrame(a).to_csv("data_preds.csv")

In [ ]:
!ls

data_preds.csv	sample_data  test_shrutime.csv	train_shrutime.csv
